In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
# import geopandas as gpd

## Cargar poblacion sintetica por circuitos, sin resamplear

In [2]:
circuito_pers = pd.read_csv('./circuito_poblacion_sint.csv')
circuito_pers['circuito'] = circuito_pers['circuito'].astype(str).str.zfill(5)#.astype(str) # Aparente bug por el cual los str quedan como numeros y depues no matchea el circuito
circuito_pers.head()

VIVIENDA_REF_ID  RADIO_REF_ID  TIPVV  V01     radio  DPTO  PROV circuito  \
0          4473222         13042      1    1  60492002  6049     6    00062   
1          4473222         13042      1    1  60492002  6049     6    00062   
2          4473033         13041      1    1  60492001  6049     6    00062   
3          4473033         13041      1    1  60492001  6049     6    00062   
4          4473323         13042      1    1  60492002  6049     6    00062   

   HOGAR_REF_ID  H05     ...      P03  P05  P06  P07  P12  P08  P09  P10  \
0       4128138    2     ...       77    1    0    1    2    2    2    1   
1       4128138    2     ...       70    1    0    1    2    2    2    1   
2       4127963    1     ...       72    1    0    1    2    2    2    1   
3       4127963    1     ...       69    1    0    1    2    2    2    1   
4       4128238    1     ...       50    1    0    1    1    2    2    2   

   CONDACT  dpto_merge  
0        1        azul  
1        1        azul  
2        3        azul  
3        3        azul  
4        1        azul  

[5 rows x 38 columns]

In [3]:
# dptos info
from unidecode import unidecode
DPTO = pd.read_csv('./../EleccionesARG/data/censo_geo/DPTO.csv', ';')[['DPTO', 'NOMDPTO']]
DPTO['dpto_merge'] = [unidecode(row) for row in DPTO['NOMDPTO'].str.lower()]
DPTO['dpto_merge'] = DPTO['dpto_merge'].str.replace('25', 'veinticinco').str.replace('9 ', 'nueve ')


poblacion = pd.DataFrame(circuito_pers.groupby(['DPTO','circuito']).count().iloc[:, 1])
poblacion.columns = ['poblacion']

poblacion = poblacion.reset_index().merge(DPTO[['DPTO', 'dpto_merge']], on = 'DPTO')

poblacion.head()

DPTO circuito  poblacion dpto_merge
0  2001    00001        165  comuna 01
1  2001    00002        154  comuna 01
2  2001    00003        312  comuna 01
3  2001    00005       4052  comuna 01
4  2001    00006        245  comuna 01

In [4]:
votos = pd.read_csv('./../EleccionesARG/data/tablas_votos/VOTOS_2015_17_circ_cnt.csv', header=[0, 1], index_col = [0, 1, 2])
votos = votos['ELECTORES'].reset_index()
votos['circuito'] = votos['circuito'].str.zfill(5)
votos.head()

dpto_merger_str circuito  PROV_REF_ID  DN_2015  paso_2017  DN_2017
0     lomas de zamora    00583            2   169408     168438   206753
1               merlo    0652A            2   124717     133454   139025
2  general pueyrredon    00364            2   131245     130613   135506
3  general pueyrredon    00365            2   116626     118106   119503
4    florencio varela    00302            2   105141     111845   117519

In [5]:
### Sample con replacement. Elegir tantos electores sinteticos como electores hay en realidad
# circuito_pers

In [7]:
elec_17 = []
for i, row in votos.iterrows():
#     elec_2019?
    n_electores_2017 = row['DN_2017']
    try:
        elec_17_circuito = circuito_pers.loc[(circuito_pers.dpto_merge == row.dpto_merger_str) &
                                           (circuito_pers.circuito == row.circuito)].sample(round(.1*n_electores_2017), replace = True)
        elec_17_circuito['circuito'] = row.circuito
        elec_17 += [elec_17_circuito]
    except:
        print(row.dpto_merger_str + ', ' + row.circuito)

ezeiza, 0290B
coronel de marina l. rosales, 0248C
coronel de marina l. rosales, 0248E
coronel de marina l. rosales, 0248D
la plata, 0496E
san fernando, 0880B
la plata, 0508F
comuna 01, 00004
coronel de marina l. rosales, 0248F
tornquist, 0955B
la matanza, 0635F
la plata, 0508G
general lavalle, 0338B
coronel de marina l. rosales, 0249B
trenque lauquen, 0961D
la plata, 0496F
carlos casares, 0166A
la plata, 0504C
pergamino, 0748E
junin, 00453
junin, 00449
junin, 00455
necochea, 00689


In [10]:
electores_17 = pd.concat(elec_17)
electores_17.to_csv('sintetico_2017.1.csv', index = False)

In [ ]:
electores_17.shape

In [ ]:
electores_17.groupby(['PROV', 'DPTO'])['PERSONA_REF_ID'].nunique()

In [ ]:
len(electores_17)

In [ ]:
electores_17.groupby(['DPTO', 'dpto_merge']).count()